<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import pandas as pd

In [2]:
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [3]:
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn

In [4]:
data = pd.read_csv("C:\\Users\\nsuguru\\Desktop\\data.csv")
from sklearn.model_selection import train_test_split
X=data.loc[:,'f1':'f2']
Y=data.loc[:,'label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, stratify=Y)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(15000, 2)
(5000, 2)
(15000,)
(5000,)


In [5]:
class LossHistory(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        self.history={'loss': [],'acc': [],'val_loss': [],'val_acc': []}
        print('train_begin')
        
    def on_epoch_end(self, epoch, logs={}):
        print('\n','epoch_end')
        # predict probabilities for test set
        yhat_probs = model.predict(X_test, verbose=0)
        # predict crisp classes for test set
        #yhat_classes = model.predict_proba(X_test, verbose=0)
        # reduce to 1d array
        yhat_probs = yhat_probs[:, 0]
        #yhat_classes = yhat_classes[:, 0]
        #f1 = f1_score(Y_test, yhat_classes)
        #print('F1 score: %f' % f1)
        auc = roc_auc_score(Y_test, yhat_probs)
        print('ROC AUC: %f' % auc)
        ## on end of each epoch, we will get logs and update the self.history dict
        self.history['loss'].append(logs.get('loss'))
        self.history['acc'].append(logs.get('acc'))
        #self.history['auc'].append(logs.get('auc'))
        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_acc', -1) != -1:
            self.history['val_acc'].append(logs.get('val_acc'))

In [12]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=20))(input_layer)
layer2 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=30))(layer1)
layer3 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=40))(layer2)
layer4 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=50))(layer3)
layer5 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=60))(layer4)

#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=70))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)


#Callbacks
history_own = LossHistory()

optimizer = tf.keras.optimizers.SGD(0.01)

model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

model.fit(X_train,Y_train,epochs=100, validation_data=(X_test,Y_test), batch_size=1000, callbacks=[history_own])


Train on 15000 samples, validate on 5000 samples
train_begin
Epoch 1/100
 1000/15000 [=>............................] - ETA: 1s - loss: 5.6971 - acc: 0.4780
 epoch_end
ROC AUC: 0.480002
15000/15000 [==============================] - 0s 18us/sample - loss: 4.9778 - acc: 0.5013 - val_loss: 4.6114 - val_acc: 0.4816
Epoch 2/100
 1000/15000 [=>............................] - ETA: 0s - loss: 4.7474 - acc: 0.4970
 epoch_end
ROC AUC: 0.478902
15000/15000 [==============================] - 0s 5us/sample - loss: 4.4374 - acc: 0.4950 - val_loss: 4.1854 - val_acc: 0.4798
Epoch 3/100
 1000/15000 [=>............................] - ETA: 0s - loss: 4.2824 - acc: 0.4860
 epoch_end
ROC AUC: 0.519658
15000/15000 [==============================] - 0s 6us/sample - loss: 3.9386 - acc: 0.4923 - val_loss: 3.6516 - val_acc: 0.5000
Epoch 4/100
 1000/15000 [=>............................] - ETA: 0s - loss: 4.0293 - acc: 0.4840
 epoch_end
ROC AUC: 0.520936
15000/15000 [==============================] - 0s 5us/sam

 1000/15000 [=>............................] - ETA: 0s - loss: 0.6907 - acc: 0.5320
 epoch_end
ROC AUC: 0.505999
15000/15000 [==============================] - 0s 5us/sample - loss: 0.6907 - acc: 0.5053 - val_loss: 0.6934 - val_acc: 0.5060
Epoch 34/100
 1000/15000 [=>............................] - ETA: 0s - loss: 0.6918 - acc: 0.4580
 epoch_end
ROC AUC: 0.505999
15000/15000 [==============================] - 0s 4us/sample - loss: 0.6906 - acc: 0.5054 - val_loss: 0.6934 - val_acc: 0.5060
Epoch 35/100
 1000/15000 [=>............................] - ETA: 0s - loss: 0.6924 - acc: 0.5140
 epoch_end
ROC AUC: 0.505999
15000/15000 [==============================] - 0s 5us/sample - loss: 0.6906 - acc: 0.5053 - val_loss: 0.6934 - val_acc: 0.5060
Epoch 36/100
 1000/15000 [=>............................] - ETA: 0s - loss: 0.6936 - acc: 0.5310
 epoch_end
ROC AUC: 0.505999
15000/15000 [==============================] - 0s 5us/sample - loss: 0.6906 - acc: 0.5053 - val_loss: 0.6934 - val_acc: 0.5060
E

Epoch 66/100
 1000/15000 [=>............................] - ETA: 0s - loss: 0.6918 - acc: 0.5050
 epoch_end
ROC AUC: 0.507599
15000/15000 [==============================] - 0s 4us/sample - loss: 0.6906 - acc: 0.5059 - val_loss: 0.6932 - val_acc: 0.5068
Epoch 67/100
 1000/15000 [=>............................] - ETA: 0s - loss: 0.6908 - acc: 0.5310
 epoch_end
ROC AUC: 0.507599
15000/15000 [==============================] - 0s 4us/sample - loss: 0.6906 - acc: 0.5061 - val_loss: 0.6932 - val_acc: 0.5072
Epoch 68/100
 1000/15000 [=>............................] - ETA: 0s - loss: 0.6934 - acc: 0.5290
 epoch_end
ROC AUC: 0.507599
15000/15000 [==============================] - 0s 4us/sample - loss: 0.6906 - acc: 0.5059 - val_loss: 0.6932 - val_acc: 0.5072
Epoch 69/100
 1000/15000 [=>............................] - ETA: 0s - loss: 0.6898 - acc: 0.5180
 epoch_end
ROC AUC: 0.507599
15000/15000 [==============================] - 0s 4us/sample - loss: 0.6905 - acc: 0.5061 - val_loss: 0.6931 - val_

ROC AUC: 0.525352
15000/15000 [==============================] - 0s 4us/sample - loss: 0.6870 - acc: 0.5240 - val_loss: 0.6884 - val_acc: 0.5244
Epoch 99/100
 1000/15000 [=>............................] - ETA: 0s - loss: 0.6990 - acc: 0.5360
 epoch_end
ROC AUC: 0.526395
15000/15000 [==============================] - 0s 4us/sample - loss: 0.6867 - acc: 0.5253 - val_loss: 0.6882 - val_acc: 0.5264
Epoch 100/100
 1000/15000 [=>............................] - ETA: 0s - loss: 0.6810 - acc: 0.5410
 epoch_end
ROC AUC: 0.526416
15000/15000 [==============================] - 0s 5us/sample - loss: 0.6866 - acc: 0.5252 - val_loss: 0.6881 - val_acc: 0.5260


In [14]:
history_own.history

{'loss': [7.6666240976817575,
  7.6666239339913895,
  7.666623891289555,
  7.666623898406527,
  7.666623912640472],
 'acc': [0.5, 0.5, 0.5, 0.5, 0.5],
 'val_loss': [7.6666239969658125,
  7.6666239536169805,
  7.6666239102681475,
  7.6666239536169805,
  7.666623866919315],
 'val_acc': [0.5, 0.5, 0.5, 0.5, 0.5]}

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 6         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 3     

In [51]:
tf.compat.v1.disable_eager_execution()

In [7]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(10, input_dim=2, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

history_own = LossHistory()
#optimizer = tf.keras.optimizers.Adam(0.001)
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=200, validation_data=(X_test,Y_test), batch_size=1000, callbacks=[history_own])

Using TensorFlow backend.
W1130 12:56:16.314110 10824 deprecation_wrapper.py:119] From C:\Users\nsuguru\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1130 12:56:16.315103 10824 deprecation_wrapper.py:119] From C:\Users\nsuguru\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



RuntimeError: tf.placeholder() is not compatible with eager execution.